# Advent of Code 2021 - Day 16

In [202]:
sealed interface Packet {
    val version: Binary
    val type: Binary
}
data class HeaderPacket(override val version: Binary, override val type: Binary, val rest: String) : Packet

sealed interface ConcretePacket : Packet
data class LiteralPacket(override val version: Binary, override val type: Binary, val value: Binary) : ConcretePacket
data class OperatorPacket(override val version: Binary, override val type: Binary, val packets: List<ConcretePacket>) : ConcretePacket

data class Binary(val bits: String) : CharSequence by bits {
    val int: Long = bits.toLong(2)
    override fun toString() = "Binary($int:$bits)"
}

In [203]:
import java.io.File
import java.util.Scanner
import kotlin.sequences.generateSequence

val input: List<String> = File("Day16.input.txt")
    .bufferedReader()
    .lineSequence()
    .map(String::asSequence)
    .map { line -> line.joinToString("") { it.digitToInt(16).toString(2).padStart(4, '0') } }
    .toList()

## Part 1

This problem is way too long to attempt to summarize. Visit the site for the problem statement.

In [204]:
fun HeaderPacket.drop(n: Int): HeaderPacket = copy(rest = rest.drop(n))
operator fun Binary.minus(value: Int): Binary = Binary((int - value).toString(2))
operator fun Binary.plus(value: Long): Binary = Binary((int + value).toString(2))

fun String.headerPacket(): HeaderPacket? = chunked(3).let {
    if (it.size < 3) null
    else HeaderPacket(
        version = Binary(it[0]),
        type = Binary(it[1]),
        rest = it.drop(2).joinToString("")
    )
}

fun HeaderPacket.process(): Pair<ConcretePacket, String> = when (type.int) {
    4L -> literal()
    else -> operator()
}

fun HeaderPacket.literal(): Pair<LiteralPacket, String> {
    require(type.int == 4L)
    var end = false
    return rest.chunked(5)
        .takeWhile { window -> !end.also { end = window[0] == '0' } }
        .map { it.drop(1) }
        .let {
            LiteralPacket(
                version = version,
                type = type,
                value = Binary(it.joinToString("")), 
            ) to rest.drop(5 * it.size)
         }
}

fun HeaderPacket.operator(): Pair<OperatorPacket, String> {
    require(type.int != 4L)
    fun HeaderPacket.operatorZero(): Pair<OperatorPacket, String> {
        var length = Binary(rest.take(15))
        var processing = rest.drop(15)
        val packets = mutableListOf<ConcretePacket>()
        while (length.int > 0) {
            processing.headerPacket()?.process()?.let {
                packets.add(it.first)
                length -= processing.removeSuffix(it.second).length
                processing = it.second
            } ?: TODO()
        }
        
        return OperatorPacket(
            version = version,
            type = type,
            packets = packets,
        ) to processing
    }

    fun HeaderPacket.operatorOne(): Pair<OperatorPacket, String> {
        var length = Binary(rest.take(11))
        var processing = rest.drop(11)
        val packets = mutableListOf<ConcretePacket>()
        while (length.int > 0) {
            processing.headerPacket()?.process()?.let {
                packets.add(it.first)
                length -= 1
                processing = it.second
            } ?: TODO()
        }
        
        return OperatorPacket(
            version = version,
            type = type,
            packets = packets,
        ) to processing
    }

    return when (rest[0]) {
        '0' -> drop(1).operatorZero()
        '1' -> drop(1).operatorOne()
        else -> TODO()
    }
}

fun String.toPacket(): Packet = headerPacket()?.process()?.first ?: TODO()

fun Packet.sum(getValue: Packet.() -> Binary): Binary {
    return when (this) {
        is OperatorPacket -> this.getValue() + packets.sumOf { it.sum(getValue).int }
        else -> this.getValue()
    }
}

input[0].toPacket().sum(Packet::version)

Binary(923:1110011011)

### Notes

This was really, really, tedious. It was also really confusing to try to figure out exactly what I was supposed to do since I misinterpreted a few critical points. I was initially thinking a recursive approach, but it enough work to just read the problem. I was thinking that maybe I could do something with `Flow` or `Sequence` since it seems like you might be able to process each bit sequentially, but also gave up on that. Lastly, I tried to implement some abstractions / guardrails and it went through a bunch of different iterations, so it could definitely use one last refactor, but this problem isn't fun.

## Part 2



In [205]:
fun Packet.operate(): Long {
    return when (type.int) {
        0L -> with(this as OperatorPacket) {
            packets.sumOf { it.operate() }
        }
        1L -> with(this as OperatorPacket) {
            packets.fold(1L) { acc, curr -> curr.operate() * acc }
        }
        2L -> with(this as OperatorPacket) {
            packets.map { it.operate() }.minByOrNull { it }!!
        }
        3L -> with(this as OperatorPacket) {
            packets.map { it.operate() }.maxByOrNull { it }!!
        }
        4L -> with(this as LiteralPacket) {
            value.int
        }
        5L -> with(this as OperatorPacket) {
            if (packets[0].operate() > packets[1].operate()) 1L else 0L 
        }
        6L -> with(this as OperatorPacket) {
            if (packets[0].operate() < packets[1].operate()) 1L else 0L 
        }
        7L -> with(this as OperatorPacket) {
            if (packets[0].operate() == packets[1].operate()) 1L else 0L 
        }
        else -> TODO()
    }
}

input[0].toPacket().operate()

258888628940

### Notes

In an ideal scenario, I'd probably go back and refactor once again with this new requirement. I would probably have specific implementation for each `OperatorPacket` like `SumOperatorPacket`, `Product...`, etc. I'd probably make sure that each class implements their own `operate` function. But again, the problem wasn't a lot of fun.